# Recurrent Neural Networks

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/tensorflow/05-tensorflow-cnn.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)

---

## Introduction

In this notebook, we will explore **recurrent neural networks**
(RNNs) and see their application in modeling sequential data. TODO

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import tensorflow.keras as kr
from tensorflow.keras.datasets import cifar10


from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

import warnings
warnings.simplefilter(action="once")

from pathlib import Path
DATASET_DIR = Path("./data").absolute()

seed = 0
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

print(tf.config.list_physical_devices())
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.8.0
